# Convert collimator tables in json files

## Imports

In [2]:
import json
import re
import os
from pathlib import Path

### Open and read input file

In [3]:
# === Input and output file paths ===
#example_folder = os.getcwd()
parent_folder = Path.cwd().parent
input_path = os.path.join(parent_folder,'dat_files/FCCee_z_GHC_V23_tcp_tcs.dat')
output_path = input_path.replace("dat_files", "json_files").replace(".dat", ".json")

# === Open and read the input file ===
with open(input_path, "r") as file:
    lines = file.readlines()

### Prapare the JSON structure

In [4]:
# === Prepare the JSON structure ===
collimator_data = {}

for line in lines[1:]:  # Skip header
    parts = re.split(r'\s{2,}', line.strip())
    if len(parts) < 13:
        continue  # Not enough fields

    try:
        name = parts[0]
        plane = parts[2]     
        s = float(parts[4])
        length = float(parts[5])
        n_sigma = float(parts[6])
        material_raw = parts[9].lower()
    except (ValueError, IndexError):
        continue  # Skip malformed rows

    # Normalize material names
    material_map = {
        'mogr': 'moc',
        'mo': 'Mo',
    }
    material = material_map.get(material_raw, material_raw)

    angle = 0.0 if plane.upper() == 'H' else 1.57079633

    collimator_data[name] = {
        "s": s,
        "angle": angle,
        "layers": [{"material": material, "thickness": 10}],
        "n_sigma": n_sigma,
        "length": length,
        "tilt_1": 0.0,
        "tilt_2": 0.0,
        "halfgap_dependent_hom_files": []
    }

### Save JSON to a file

In [5]:
# === Save JSON to a file ===
with open(output_path, "w") as json_file:
    json.dump(collimator_data, json_file, indent=2)

print(f"✅ JSON saved to {output_path} with {len(collimator_data)} entries")

✅ JSON saved to /home/dogibell/fcc_ee_pywit_model/CEI_section_meeting/2_Collimation_Optics/json_files/FCCee_z_GHC_V23_tcp_tcs.json with 9 entries
